<a href="https://colab.research.google.com/github/SampathK/MyExperimentalNotebooks/blob/main/Large_Text_Summarization_Using_AWS_BedRock_Mistral_Large.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%sh
pip install -Uq boto3 langchain langchain_community tiktoken

In [8]:
file_path = "./input/meeting_1.txt"

# Open the file in read mode and read its contents into a string
with open(file_path, "r") as file:
    call_transcripts = file.read()

In [17]:
import re

def preprocess_transcript(transcript):
    # Remove timestamps (assuming the format is [hh:mm:ss])
    transcript = re.sub(r'\[\d{2}:\d{2}:\d{2}\]', '', transcript)

    # Remove extra whitespace
    transcript = re.sub(r'\s+', ' ', transcript).strip()

    # Remove any unnecessary special characters (if needed)
    transcript = re.sub(r'[^\w\s.,?!:]', '', transcript)

    transcript = re.sub(r"<.*>", "", transcript)
    transcript = re.sub(r"\[(.*)\]", "", transcript)
    transcript = re.sub(r"\n\n\n", "\n", transcript)

    return transcript

In [18]:
processed_transcript = preprocess_transcript(call_transcripts)
print(processed_transcript[:1000])

Sure! Here is a larger conversation document simulating a detailed client meeting that can be split into multiple chunks:   Emily: Good afternoon, everyone. Thank you for joining todays meeting. Im Emily, and Ill be moderating our session. Just a reminder, this meeting is being recorded. Lets start with introductions. John: Hi, everyone. Im John, the VP of Marketing at Acme Corp. Looking forward to discussing our upcoming campaign strategies. Jane: Hello, everyone. Im Jane, a marketing consultant from Creative Solutions. Im excited to dive into the details and help shape your campaign. Michael: Hi, Im Michael, the project manager from Acme Corp. Ill be overseeing the implementation of our marketing campaigns. Sarah: Hi, Im Sarah, the data analyst from Acme Corp. Ill be providing insights and analytics to support our strategies. Emily: Great! Thanks, everyone. Lets get started. John, could you give us an overview of the objectives for this meeting? John: Absolutely. Were here to discuss

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_text(text, chunk_size, chunk_overlap):
    """
    Splits a text into smaller chunks based on the specified chunk size and overlap.

    Args:
    - text (str): The text to be split.
    - chunk_size (int): The maximum size of each chunk.
    - chunk_overlap (int): The number of characters to overlap between chunks.

    Returns:
    - list: A list of split texts.

    Raises:
    - ValueError: If the chunk size or overlap is invalid.
    """
    if not isinstance(text, str):
        raise ValueError("Text must be a string.")
    if not isinstance(chunk_size, int) or chunk_size <= 0:
        raise ValueError("Chunk size must be a positive integer.")
    if not isinstance(chunk_overlap, int) or chunk_overlap < 0:
        raise ValueError("Chunk overlap must be a non-negative integer.")
    if chunk_overlap >= chunk_size:
        raise ValueError("Chunk overlap must be less than chunk size.")

    # Create a RecursiveCharacterTextSplitter instance
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,  # Set the chunk size
        chunk_overlap=chunk_overlap,  # Set the chunk overlap
        separators=['\n', ' ', '\n\n'],  # Define the separators
        keep_separator=False  # Do not keep the separators
    )

    # Split the text using the RecursiveCharacterTextSplitter
    chunks = splitter.split_text(text)

    return chunks

In [182]:
# Example usage
text =  processed_transcript
print(type(text))
chunk_size = 4096
chunk_overlap = 1000

splitted_text = split_text(text, chunk_size, chunk_overlap)

print("Splitted Text:")
for i, chunk in enumerate(splitted_text):
    type(text)
    print(type(chunk))
    print(f"Chunk {i+1}: {chunk}")

<class 'str'>
Splitted Text:
<class 'str'>
Chunk 1: Sure! Here is a larger conversation document simulating a detailed client meeting that can be split into multiple chunks: Emily: Good afternoon, everyone. Thank you for joining todays meeting. Im Emily, and Ill be moderating our session. Just a reminder, this meeting is being recorded. Lets start with introductions. John: Hi, everyone. Im John, the VP of Marketing at Acme Corp. Looking forward to discussing our upcoming campaign strategies. Jane: Hello, everyone. Im Jane, a marketing consultant from Creative Solutions. Im excited to dive into the details and help shape your campaign. Michael: Hi, Im Michael, the project manager from Acme Corp. Ill be overseeing the implementation of our marketing campaigns. Sarah: Hi, Im Sarah, the data analyst from Acme Corp. Ill be providing insights and analytics to support our strategies. Emily: Great! Thanks, everyone. Lets get started. John, could you give us an overview of the objectives for th

In [184]:
import json
from google.colab import userdata
import boto3
from botocore.exceptions import ClientError
client_bedrock = boto3.client("bedrock-runtime", "us-east-1", aws_access_key_id=userdata.get("AWS_SERVER_PUBLIC_KEY"),
    aws_secret_access_key=userdata.get("AWS_SERVER_SECRET_KEY"),)

In [185]:
def create_summary(prompt):
    try:
        prompt = f"<s>[INST]{prompt} [/INST]"
        body = json.dumps({
            "temperature": 0.2,
            "max_tokens": 8192,
            "prompt": prompt,
            "top_p": 1,
            "top_k": 200,
            "stop":["</s>"]

        })
        #body = "{\"prompt\":\"<s>[INST] this is where you place your input text [/INST]\", \"max_tokens\":200, \"temperature\":0.5, \"top_p\":0.9, \"top_k\":50}"
        #body = "{\"prompt\":\"<s>[INST] this is where you place your input text [/INST]\", \"max_tokens\":4096, \"temperature\":0.2, \"top_p\":1, \"top_k\":200}"
        #body =  "{\"prompt\":\"<s>[INST] this is where you place your input text [/INST]\", \"max_tokens\": 4096, \"temperature\": 0.2, \"top_p\": 1, \"top_k\": 200}"
        #body =  "{\"prompt\": \"<s>[INST] <replace_me> [/INST]\",  \"max_tokens\": 8192, \"temperature\": 0.2, \"top_p\": 1, \"top_k\": 200}"
        #print(f"Request body: {body}")
        accept = "application/json"
        content_type = "application/json"

        response = client_bedrock.invoke_model(
            body=body,
            modelId="mistral.mistral-large-2402-v1:0",
            accept=accept,
            contentType=content_type,
        )
        response_body = json.loads(response.get("body").read())
        #print(f"Response body: {response_body}")

        # remove the first line of text that explains the task completed
        # e.g. " Here are three hypothetical questions that the passage could help answer:\n\n"
        formatted_response = ""
        try:
            formatted_response = (
                response_body['outputs'][0]['text'].split("\n", 2)[2].strip()
            )
        except:
            formatted_response = response_body['outputs'][0]['text']
        return formatted_response
    except ClientError as ex:
        print(ex)
        exit(1)

In [186]:
text = splitted_text[1].strip()

In [192]:
prompt = f"""<text>{text}</text>
Please summarize the above meeting transcript within text XML into a comprehensive summary, including all key details. The summary should include the following fields:

1. Participants: List all unique participants or human or owner names mentioned in the meeting.
2. Summary: Provide a brief overview of the main topics discussed during the meeting.
3. Key Points: Extract the most significant points discussed.
4. Decisions Made: List all decisions made during the meeting.
5. Action Items: Include detailed action items with descriptions, owners, and deadlines.
6. Strengths: Capture any positive aspects or strengths highlighted during the meeting.
7. Weaknesses: Capture any negative aspects or weaknesses highlighted during the meeting.
8. Progress Updates: Provide updates on previous action items and overall project progress.
9. Meeting Date and Time: Record the date and time when the meeting took place.
10. Meeting Duration: Specify the duration of the meeting.
11. Meeting Type: Categorize the type of meeting (e.g., planning, review, status update).
12. Sentiment Scores: Provide numeric scores for positive and negative sentiments.
13. Tags: Include keywords or tags relevant to the meeting content.
14. Project Name: Associate the meeting with a specific project.
15. Location: Specify the location (physical or virtual) of the meeting.
The final summary should be in JSON format, structured as follows:
{{
  "participants": ["Participant 1", "Participant 2", "Participant 3", ...],
  "summary": "Brief overview of the meeting and main topics discussed.",
  "key_points": [
    "Key point 1",
    "Key point 2",
    "Key point 3",
    ...
  ],
  "decisions_made": [
    "Decision 1",
    "Decision 2",
    "Decision 3",
    ...
  ],
  "action_items": [
    {{"description": "Action item 1", "owner": "Person 1", "deadline": "YYYY-MM-DD"}},
    {{"description": "Action item 2", "owner": "Person 2", "deadline": "YYYY-MM-DD"}},
    ...
  ],
  "strengths": [
    "Positive aspect 1",
    "Positive aspect 2",
    ...
  ],
  "weaknesses": [
    "Negative aspect 1",
    "Negative aspect 2",
    ...
  ],
  "progress_updates": [
    "Update on previous action item 1",
    "Update on previous action item 2",
    ...
  ],
  "meeting_date": "YYYY-MM-DDTHH:MM:SSZ",
  "meeting_duration": 60,
  "meeting_type": "Planning",
  "positive_sentiment_score": 0.8,
  "negative_sentiment_score": 0.1,
  "tags": ["tag1", "tag2", "tag3"],
  "project_name": "Project Name",
  "location": "Location"
}}
"""
print(create_summary(prompt))

"summary": "The team discussed updates on the influencer collaboration, email campaigns, and SEO improvements. They also addressed concerns about social media engagement and email templates.",
  "key_points": [
    "Received responses from 3 out of 5 influencers, with 2 interested in collaboration",
    "Improvement in email campaign performance due to new templates",
    "Significant improvement in site speed and gradual increase in organic search traffic",
    "Concern about plateaued social media engagement",
    "Suggestion to tweak social media content strategy and run targeted ads"
  ],
  "decisions_made": [
    "Move forward with the 2 interested influencers",
    "Prioritize the redesign of email templates",
    "Adjust social media content strategy and plan targeted ads"
  ],
  "action_items": [
    {"description": "Finalize details with the 2 interested influencers", "owner": "Jane", "deadline": "Not specified"},
    {"description": "Redesign email templates", "owner": "Micha

In [188]:
import datetime
summary = ""
print(f"Start time: {datetime.datetime.now()}")

for i, text in enumerate(splitted_text[0::]):
    try:
        prompt = f"""<text>{text}</text>
Please summarize the above meeting transcript within text XML into a comprehensive summary, including all key details. The summary should include the following fields:

1. Participants: List all participants mentioned in the meeting.
2. Summary: Provide a brief overview of the main topics discussed during the meeting.
3. Key Points: Extract the most significant points discussed.
4. Decisions Made: List all decisions made during the meeting.
5. Action Items: Include detailed action items with descriptions, owners, and deadlines.
6. Strengths: Capture any positive aspects or strengths highlighted during the meeting.
7. Weaknesses: Capture any negative aspects or weaknesses highlighted during the meeting.
8. Progress Updates: Provide updates on previous action items and overall project progress.
9. Meeting Date and Time: Record the date and time when the meeting took place.
10. Meeting Duration: Specify the duration of the meeting.
11. Meeting Type: Categorize the type of meeting (e.g., planning, review, status update).
12. Sentiment Scores: Provide numeric scores for positive and negative sentiments.
13. Tags: Include keywords or tags relevant to the meeting content.
14. Project Name: Associate the meeting with a specific project.
15. Location: Specify the location (physical or virtual) of the meeting.
The final summary should be in JSON format, structured as follows:
{{
  "participants": ["Participant 1", "Participant 2", "Participant 3", ...],
  "summary": "Brief overview of the meeting and main topics discussed.",
  "key_points": [
    "Key point 1",
    "Key point 2",
    "Key point 3",
    ...
  ],
  "decisions_made": [
    "Decision 1",
    "Decision 2",
    "Decision 3",
    ...
  ],
  "action_items": [
    {{"description": "Action item 1", "owner": "Person 1", "deadline": "YYYY-MM-DD"}},
    {{"description": "Action item 2", "owner": "Person 2", "deadline": "YYYY-MM-DD"}},
    ...
  ],
  "strengths": [
    "Positive aspect 1",
    "Positive aspect 2",
    ...
  ],
  "weaknesses": [
    "Negative aspect 1",
    "Negative aspect 2",
    ...
  ],
  "progress_updates": [
    "Update on previous action item 1",
    "Update on previous action item 2",
    ...
  ],
  "meeting_date": "YYYY-MM-DDTHH:MM:SSZ",
  "meeting_duration": 60,
  "meeting_type": "Planning",
  "positive_sentiment_score": 0.8,
  "negative_sentiment_score": 0.1,
  "tags": ["tag1", "tag2", "tag3"],
  "project_name": "Project Name",
  "location": "Location"
}}
"""
        chapter_summary = create_summary(prompt)
        chapter_summary = f"\nPart  {i + 1}:\n{(chapter_summary)}\n\n"
        summary += chapter_summary
        print(f"Part {i + 1}/{len(splitted_text)} completed...", end="\r")
    except Exception as ex:
        chapter_summary = f"Part {i + 1}/{len(splitted_text)} failed: {ex}"
        print(chapter_summary)
        summary += chapter_summary

print(f"Finish time: {datetime.datetime.now()}")

Start time: 2024-06-23 10:00:19.239929
Finish time: 2024-06-23 10:01:02.472951


In [189]:
print(summary)


Part  1:
"summary": "Discussion on Acme Corp's marketing campaign for the next quarter, focusing on increasing online presence and driving traffic to the website.",
  "key_points": [
    "Acme Corp aims to boost online presence and website traffic through social media marketing, SEO, and email campaigns.",
    "Website traffic has declined in the past two months, social media engagement is inconsistent, and email clickthrough rates need improvement.",
    "Jane suggested creating interactive social media content, segmenting email audiences, and optimizing website content for SEO."
  ],
  "decisions_made": [
    "Implement interactive content for social media.",
    "Segment email audience and tailor content.",
    "Optimize website content for SEO.",
    "Collaborate with influencers to expand reach."
  ],
  "action_items": [
    {"description": "Create social media content calendar", "owner": "Jane", "deadline": "2022-04-30"},
    {"description": "Identify and reach out to potential 

In [190]:
final_summary = summary
if len(splitted_text) > 1:
    prompt = f"""<summaries>{summary}</summaries>
Please summarize the above client meeting transcript summaries within summaries XML tag of a document into a concise, comprehensive consolidated summary. Make sure to include the key points, decisions made, action items, positive and negative sentiments, progress updates, and names of participants mentioned in the summaries.
If it is not possible to create a single summary without losing important details due to length constraints, provide the summaries as they are.
The summary should include:
1. Participants: List all unique participants mentioned in the meeting.
2. Summary: Provide a brief overview of the unique main topics discussed during the meeting.
3. Key Points: Extract the most significant unique points discussed.
4. Decisions Made: List all unique decisions made during the meeting.
5. Action Items: Include detailed unique action items with descriptions, owners, and deadlines.
6. Strengths: Capture any unique positive aspects or strengths highlighted during the meeting.
7. Weaknesses: Capture any unique negative aspects or weaknesses highlighted during the meeting.
8. Progress Updates: Provide unique updates on previous action items and overall project progress.
9. Meeting Date and Time: Record the date and time when the meeting took place.
10. Meeting Duration: Specify the duration of the meeting.
11. Meeting Type: Categorize the type of meeting (e.g., planning, review, status update).
12. Sentiment Scores: Provide overall numeric scores for positive and negative sentiments.
13. Tags: Include unique keywords or tags relevant to the meeting content.
14. Project Name: Associate the meeting with a specific project.
15. Location: Specify the location (physical or virtual) of the meeting.
The final summary should be in JSON format, structured as follows:
{{
  "participants": ["Participant 1", "Participant 2", "Participant 3", ...],
  "summary": "Brief overview of the meeting and main topics discussed.",
  "key_points": [
    "Key point 1",
    "Key point 2",
    "Key point 3",
    ...
  ],
  "decisions_made": [
    "Decision 1",
    "Decision 2",
    "Decision 3",
    ...
  ],
  "action_items": [
    {{"description": "Action item 1", "owner": "Person 1", "deadline": "YYYY-MM-DD"}},
    {{"description": "Action item 2", "owner": "Person 2", "deadline": "YYYY-MM-DD"}},
    ...
  ],
  "strengths": [
    "Positive aspect 1",
    "Positive aspect 2",
    ...
  ],
  "weaknesses": [
    "Negative aspect 1",
    "Negative aspect 2",
    ...
  ],
  "progress_updates": [
    "Update on previous action item 1",
    "Update on previous action item 2",
    ...
  ],
  "meeting_date": "YYYY-MM-DDTHH:MM:SSZ",
  "meeting_duration": 60,
  "meeting_type": "Planning",
  "positive_sentiment_score": 0.8,
  "negative_sentiment_score": 0.1,
  "tags": ["tag1", "tag2", "tag3"],
  "project_name": "Project Name",
  "location": "Location"
  }}
    """
    final_summary = create_summary(prompt)

In [191]:
print(final_summary)

"summary": "The meetings focused on discussing and implementing marketing strategies for Acme Corp's Q2 campaign, including social media, influencer collaboration, email campaigns, and SEO. Progress was reviewed, decisions were made, and action items were set.",
  "key_points": [
    "Acme Corp aims to boost online presence and website traffic through social media marketing, SEO, and email campaigns.",
    "The team plans to create interactive social media content and collaborate with influencers.",
    "Email campaigns will be personalized using data segmentation.",
    "An SEO audit has been conducted, and improvements are being implemented.",
    "Received responses from 3 out of 5 influencers, with 2 interested in collaboration",
    "Improved email campaign performance with new templates",
    "Significant improvement in site speed and gradual increase in organic search traffic",
    "Plateaued social media engagement and the need to tweak content strategy"
  ],
  "decisions_made"